# Iptables

> Iptables

- skip_showdoc: true
- skip_exec: true

## Basic Concepts

### Tables

> iptables organizes rules into different tables. Each table contains a set of chains:

- Filter Table: The default table for filtering packets. It has three built-in chains: INPUT, FORWARD, and OUTPUT.
- NAT Table: Used for Network Address Translation (NAT). It has three built-in chains: PREROUTING, POSTROUTING, and OUTPUT.
- Mangle Table: Used for specialized packet alterations. It has five built-in chains: PREROUTING, INPUT, FORWARD, OUTPUT, and POSTROUTING.
- Raw Table: Used for configuring exemptions from connection tracking. It has two built-in chains: PREROUTING and OUTPUT.

### Chains

> Chains are lists of rules that iptables uses to determine the fate of packets:

- INPUT: Handles packets destined for the local machine.
- FORWARD: Handles packets routed through the machine.
- OUTPUT: Handles packets originating from the local machine.

### Rules

> Each rule in a chain specifies conditions for matching packets and an action to take (e.g., ACCEPT, DROP).

## Basic Commands

> Here are some common iptables commands

### View Existing Rules

> To list rules in a specific table

```sh
sudo iptables -L -t filter
sudo iptables -L -t nat
sudo iptables -L -t mangle


```

> To list rules with extended information
```sh
sudo iptables -L -v -n
```

### Adding Rules

> To append a rule to a chain

```sh
sudo iptables -A INPUT -p tcp --dport 22 -j ACCEPT

```
This rule allows incoming TCP traffic on port 22 (SSH)

> To insert a rule at a specific position
```sh
sudo iptables -I INPUT 1 -p tcp --dport 80 -j ACCEPT

```
This rule inserts a new rule at position 1 of the INPUT chain allowing incoming HTTP traffic.

### Deleting Rules

> To delete a specific rule
```sh
sudo iptables -D INPUT -p tcp --dport 22 -j ACCEPT
```

> To delete a rule by its line number
```sh
sudo iptables -D INPUT 1

```

### Flushing Rules

> To remove all rules from a specific table

```sh
sudo iptables -F -t filter
sudo iptables -F -t nat
sudo iptables -F -t mangle

```

### Saving Rules

> To save rules so they persist across reboots, you can use
```sh
sudo iptables-save > /etc/iptables/rules.v4
```
> To restore saved rules
```sh
sudo iptables-restore < /etc/iptables/rules.v4

```

## Common Use Cases

### Basic Firewall Setup

> To set up a basic firewall that allows only SSH and HTTP traffic

```sh
# Flush existing rules
sudo iptables -F

# Allow loopback traffic
sudo iptables -A INPUT -i lo -j ACCEPT

# Allow incoming SSH and HTTP
sudo iptables -A INPUT -p tcp --dport 22 -j ACCEPT
sudo iptables -A INPUT -p tcp --dport 80 -j ACCEPT

# Drop all other incoming traffic
sudo iptables -P INPUT DROP

```

### NAT Configuration

> To set up NAT (e.g., for a home router)

```sh
# Enable IP forwarding
sudo sysctl -w net.ipv4.ip_forward=1

# Configure NAT for outgoing traffic
sudo iptables -t nat -A POSTROUTING -o eth0 -j MASQUERADE

```

### Port Forwarding

> To forward traffic from port 8080 on the local machine to port 80 on a remote server

```sh
sudo iptables -t nat -A PREROUTING -p tcp --dport 8080 -j DNAT --to-destination 192.168.1.100:80
sudo iptables -A FORWARD -p tcp -d 192.168.1.100 --dport 80 -j ACCEPT

```

## Advanced Features

### Connection Tracking


> iptables can track the state of connections

```sh
sudo iptables -A INPUT -m conntrack --ctstate ESTABLISHED,RELATED -j ACCEPT

```
This rule allows incoming traffic that is part of an established or related connection.

### Rate Limiting

> To limit the rate of incoming connections

```sh
sudo iptables -A INPUT -p tcp --dport 22 -m limit --limit 2/minute -j ACCEPT

```
This rule limits SSH connections to 2 per minute.

### Logging

> To log packets that match a rule

```sh
sudo iptables -A INPUT -p tcp --dport 22 -j LOG --log-prefix "SSH Access: "

```
Logs will appear in `/var/log/syslog` or `/var/log/messages`, depending on your system

## Configuration Files

`iptables` rules are typically saved in `/etc/iptables/rules.v4` for IPv4 and `/etc/iptables/rules.v6` for IPv6. You can use tools like `iptables-persistent` to automatically apply saved rules on boot

```sh
sudo apt-get install iptables-persistent

```

## Transition to nftables

`nftables` is the successor to `iptables` and is intended to replace it in the future.

**Testing: Use tools like curl, telnet, or netcat to test network connectivity and rule effectiveness.**
